In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns 

# 設置 matplotlib 中文字體
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei'] # 替換為您環境支援的字體
plt.rcParams['axes.unicode_minus'] = False # 解決負號顯示問題

# ===== 0️⃣ Helper Functions (資訊熵計算) =====
def calculate_entropy(series):
    """計算資訊熵 H = -SUM(p * log2(p))，衡量多樣性"""
    series = series.dropna() 
    if series.empty:
        return 0.0
        
    counts = series.value_counts()
    probabilities = counts / len(series)
    # 使用 np.log2，計算熵值
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy


# ===== 1️⃣ Load CSV =====
# 請根據您的檔案路徑修改 dir_path
dir_path = "" 
df_txn = pd.read_csv(os.path.join(dir_path, 'acct_transaction.csv'))
df_alert = pd.read_csv(os.path.join(dir_path, 'acct_alert.csv'))
df_test = pd.read_csv(os.path.join(dir_path, 'acct_predict.csv'))

# 轉換日期格式以便比較
df_txn['txn_date'] = pd.to_numeric(df_txn['txn_date'])
df_alert['event_date'] = pd.to_numeric(df_alert['event_date'])
print("(Finish) Load Dataset.")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# ===== 2️⃣ Data Preparation & Exploration (Advanced EDA) - 數據探索 =====
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

print("\n--- 數據探索 (EDA) 結果 ---")

# 假設 df_txn, df_alert 已經載入
alert_accts = df_alert['acct'].tolist()
df_txn['is_alert'] = df_txn['from_acct'].isin(alert_accts).astype(int)

# 確保 txn_time 格式正確，並提取小時
try:
    df_txn['hour'] = pd.to_datetime(df_txn['txn_time'], format='%H:%M:%S').dt.hour
except ValueError:
    print("Warning: txn_time 格式錯誤，可能包含缺失值或非標準格式，設為 0。")
    df_txn['hour'] = 0
    
df_txn_labeled = df_txn.copy()

# -------------------------------------------------------------------
# A. 基礎 EDA (時段、渠道、金額分箱)
# -------------------------------------------------------------------

# --- A.1 時段規律探索 ---
hourly_analysis = df_txn_labeled.groupby(['is_alert', 'hour'])['txn_amt'].count().rename('txn_count').reset_index()
pivot_count = hourly_analysis.pivot_table(index='hour', columns='is_alert', values='txn_count', fill_value=0)
pivot_count.columns = ['Normal', 'Alert']
pivot_count['Alert_Ratio'] = pivot_count['Alert'] / (pivot_count['Alert'] + pivot_count['Normal'])
print("【A.1 時段探索】警示帳戶每小時交易筆數佔比 (Alert Ratio):")
print(pivot_count.sort_values(by='Alert_Ratio', ascending=False).head(8))

# --- A.2 渠道偏好探索 ---
channel_analysis = df_txn_labeled.groupby('is_alert')['channel_type'].value_counts(normalize=True).rename('proportion').reset_index()
channel_pivot = channel_analysis.pivot_table(index='channel_type', columns='is_alert', values='proportion', fill_value=0)
channel_pivot.columns = ['Normal_Prop', 'Alert_Prop']
channel_pivot['Prop_Diff'] = channel_pivot['Alert_Prop'] - channel_pivot['Normal_Prop']
print("\n【A.2 渠道探索】警示帳戶與正常帳戶的渠道使用比例差異 (Prop Diff):")
print(channel_pivot.sort_values(by='Prop_Diff', ascending=False).head(5))

# --- A.3 金額區間探索 ---
bins = [0, 1000, 5000, 10000, 50000, 100000, df_txn_labeled['txn_amt'].max() + 1]
labels = ['<1K', '1K-5K', '5K-10K', '10K-50K', '50K-100K', '>100K']
df_txn_labeled['amt_bin'] = pd.cut(df_txn_labeled['txn_amt'], bins=bins, labels=labels, right=False)
amt_bin_analysis = (
    df_txn_labeled.groupby('is_alert')['amt_bin']
    .value_counts(normalize=True).rename('proportion').reset_index()
)
amt_bin_pivot = amt_bin_analysis.pivot_table(index='amt_bin', columns='is_alert', values='proportion', fill_value=0)
amt_bin_pivot.columns = ['Normal_Prop', 'Alert_Prop']
amt_bin_pivot['Prop_Diff'] = amt_bin_pivot['Alert_Prop'] - amt_bin_pivot['Normal_Prop']
print("\n【A.3 金額區間探索】警示帳戶與正常帳戶的交易金額分佈差異 (Prop Diff):")
print(amt_bin_pivot.sort_values(by='Prop_Diff', ascending=False))


# -------------------------------------------------------------------
# B. 進階 EDA (關係、時間、幣別)
# -------------------------------------------------------------------

print("\n--- B. 進階關係與細節探索 (Advanced Features EDA) ---")

# --- B.1 交易對手類型偏好 (to_acct_type) ---
acct_type_analysis = (
    df_txn_labeled.groupby('is_alert')['to_acct_type']
    .value_counts(normalize=True).rename('proportion').reset_index()
)
acct_type_pivot = acct_type_analysis.pivot_table(
    index='to_acct_type', columns='is_alert', values='proportion', fill_value=0
)
acct_type_pivot.columns = ['Normal_Prop', 'Alert_Prop']
acct_type_pivot['Prop_Diff'] = acct_type_pivot['Alert_Prop'] - acct_type_pivot['Normal_Prop']

print("【B.1 收款方類型探索】警示帳戶對收款方類型 (玉山/他行) 的比例差異:")
# '01': 玉山； '02'：他行
print(acct_type_pivot.sort_values(by='Prop_Diff', ascending=False))
# 💡 潛在特徵：警示帳戶是否更傾向於轉出到他行 ('02')

# --- B.2 自我交易 (is_self_txn) 比例 ---
self_txn_analysis = (
    df_txn_labeled.groupby('is_alert')['is_self_txn']
    .value_counts(normalize=True).rename('proportion').reset_index()
)
self_txn_pivot = self_txn_analysis.pivot_table(
    index='is_self_txn', columns='is_alert', values='proportion', fill_value=0
)
self_txn_pivot.columns = ['Normal_Prop', 'Alert_Prop']
self_txn_pivot['Prop_Diff'] = self_txn_pivot['Alert_Prop'] - self_txn_pivot['Normal_Prop']

print("\n【B.2 自我交易探索】警示帳戶的自我交易比例 ('Y', 'N', 'UNK') 差異:")
print(self_txn_pivot.sort_values(by='Prop_Diff', ascending=False))
# 💡 潛在特徵：警示帳戶是否明顯避開自我交易或集中在某類

# --- B.3 幣別探索 ---
currency_analysis = (
    df_txn_labeled.groupby('is_alert')['currency_type']
    .value_counts(normalize=True).rename('proportion').reset_index()
)
currency_pivot = currency_analysis.pivot_table(
    index='currency_type', columns='is_alert', values='proportion', fill_value=0
)
currency_pivot.columns = ['Normal_Prop', 'Alert_Prop']
currency_pivot['Prop_Diff'] = currency_pivot['Alert_Prop'] - currency_pivot['Normal_Prop']

print("\n【B.3 幣別偏好探索】警示帳戶對不同幣別的使用比例差異:")
print(currency_pivot.sort_values(by='Prop_Diff', ascending=False).head(5))
# 💡 潛在特徵：除了 TWD 外，其他幣別（如 USD）的交易比例差異


# -------------------------------------------------------------------
# C. 帳戶層級總結（淨流動）
# -------------------------------------------------------------------

# 計算每個帳戶的交易活動（作為 From 和作為 To）
df_from = df_txn.groupby('from_acct')['txn_amt'].agg(['count', 'sum']).rename(columns={'count': 'out_count', 'sum': 'out_sum'}).reset_index()
df_to = df_txn.groupby('to_acct')['txn_amt'].agg(['count', 'sum']).rename(columns={'count': 'in_count', 'sum': 'in_sum'}).reset_index()

# 將 From 和 To 的活動合併到單一帳戶列表
all_accts = pd.DataFrame(
    list(set(df_txn['from_acct']) | set(df_txn['to_acct'])), 
    columns=['acct']
)

# 合併轉出和轉入的統計
df_acct_features_eda = all_accts.merge(
    df_from, left_on='acct', right_on='from_acct', how='left'
).drop(columns='from_acct').merge(
    df_to, left_on='acct', right_on='to_acct', how='left'
).drop(columns='to_acct').fillna(0)

# 加入警示標籤
df_acct_features_eda['is_alert'] = df_acct_features_eda['acct'].isin(alert_accts).astype(int)

# 建立平衡性特徵
df_acct_features_eda['net_flow'] = df_acct_features_eda['in_sum'] - df_acct_features_eda['out_sum']

print("\n【C. 帳戶層級探索】警示帳戶平均淨流動：")
print(df_acct_features_eda.groupby('is_alert')[['out_sum', 'in_sum', 'net_flow']].mean())

print("\n(Finish) Advanced Data Exploration and Analysis.")


--- 數據探索 (EDA) 結果 ---
【A.1 時段探索】警示帳戶每小時交易筆數佔比 (Alert Ratio):
        Normal  Alert  Alert_Ratio
hour                              
4      41786.0  180.0     0.004289
1      97965.0  373.0     0.003793
23    173223.0  650.0     0.003738
3      53453.0  200.0     0.003728
22    206588.0  739.0     0.003564
20    220829.0  768.0     0.003466
18    219566.0  755.0     0.003427
21    220104.0  737.0     0.003337

【A.2 渠道探索】警示帳戶與正常帳戶的渠道使用比例差異 (Prop Diff):
              Normal_Prop  Alert_Prop  Prop_Diff
channel_type                                    
03               0.451734    0.789716   0.337983
01               0.011567    0.015785   0.004218
99               0.000289    0.000313   0.000024
07               0.000093    0.000000  -0.000093
05               0.000792    0.000000  -0.000792


C:\Users\USER\AppData\Local\Temp\ipykernel_58584\1926616920.py:50: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  amt_bin_pivot = amt_bin_analysis.pivot_table(index='amt_bin', columns='is_alert', values='proportion', fill_value=0)



【A.3 金額區間探索】警示帳戶與正常帳戶的交易金額分佈差異 (Prop Diff):
          Normal_Prop  Alert_Prop  Prop_Diff
amt_bin                                     
<1K          0.160106    0.207158   0.047052
1K-5K        0.344947    0.353989   0.009042
5K-10K       0.130893    0.126358  -0.004536
>100K        0.052783    0.045636  -0.007147
50K-100K     0.052865    0.045323  -0.007542
10K-50K      0.258406    0.221536  -0.036870

--- B. 進階關係與細節探索 (Advanced Features EDA) ---
【B.1 收款方類型探索】警示帳戶對收款方類型 (玉山/他行) 的比例差異:
              Normal_Prop  Alert_Prop  Prop_Diff
to_acct_type                                    
2                0.456659    0.888646   0.431986
1                0.543341    0.111354  -0.431986

【B.2 自我交易探索】警示帳戶的自我交易比例 ('Y', 'N', 'UNK') 差異:
             Normal_Prop  Alert_Prop  Prop_Diff
is_self_txn                                    
UNK             0.753975    0.888646   0.134671
Y               0.032935    0.017895  -0.015041
N               0.213090    0.093459  -0.119631

【B.3 幣別偏好探索】警示帳戶對不同幣別的使用比例

In [ ]:
import pandas as pd
import numpy as np
import os

# 假設 df_txn, df_alert, df_test (df_submit), calculate_entropy 已經在環境中定義
# 注意：在 acct.ipynb 中，您使用的測試集檔案名為 df_test。
df_submit = df_test # 統一變數名稱以便於後續代碼邏輯

# ===== 3️⃣ ADVANCED Feature Engineering (基於探索結果) - 重新整理版 =====

CACHE_FILE_PATH = 'df_X_features_cache.pkl' # 定義快取檔案路徑

# --- 快取檢查邏輯 ---
if os.path.exists(CACHE_FILE_PATH):
    print(f"(Caching) 偵測到快取檔案：{CACHE_FILE_PATH}。正在載入...")
    df_X = pd.read_pickle(CACHE_FILE_PATH)
    print("(Finish) 快取載入完成。")
else:
    print("(Caching) 快取檔案未找到，正在執行完整的特徵工程...")
    
    # --- 預先處理與準備 ---
    try:
        # 確保 txn_time 處理完成，提取 hour 特徵
        if 'hour' not in df_txn.columns:
             # 假設 txn_time 是 'HH:MM:SS' 格式的字串，需調整程式碼以匹配您的實際數據類型
             # 由於原始筆記本中沒有看到 txn_time 轉換的代碼，這裡先假設它需要被處理
             # 如果 txn_time 是數值型，請調整為提取其小時部分
             df_txn['hour'] = pd.to_datetime(df_txn['txn_time'], format='%H:%M:%S').dt.hour
    except Exception as e:
        # 如果 txn_time 處理失敗，則需要使用其他方式處理時間
        print(f"Warning: txn_time 轉 hour 發生錯誤，請檢查 txn_time 格式。")
        
    # --- 3.1 實施嚴格的時間截止控制 (Temporal Cutoff) ---
    
    # 1. 找出所有警示帳戶的「最早」警示日期，作為截止日期
    acct_cutoff_map = (
        df_alert.groupby('acct')['event_date'].min()
        .reset_index().rename(columns={'event_date': 'cutoff_date'})
    )

    # 將截止日期合併到交易數據 (作為 from_acct 和 to_acct)
    df_txn_merged = df_txn.merge(
        acct_cutoff_map, left_on='from_acct', right_on='acct', how='left', suffixes=('', '_from')
    ).drop(columns=['acct_from'], errors='ignore').rename(columns={'cutoff_date': 'from_cutoff_date'})

    df_txn_merged = df_txn_merged.merge(
        acct_cutoff_map, left_on='to_acct', right_on='acct', how='left', suffixes=('', '_to')
    ).drop(columns=['acct_to'], errors='ignore').rename(columns={'cutoff_date': 'to_cutoff_date'})

    # 🌟 建議優化點 1：向量化時間截止過濾 🌟
    # 過濾交易：交易日期必須早於 from_acct **和** to_acct 的警示日期
    
    # 處理未警示帳戶：若無警示日期 (NaN)，視為截止日期無限大 (例如數據集最大日期+1)
    MAX_DATE = df_txn['txn_date'].max() + 1 
    df_txn_merged['from_cutoff_date'] = df_txn_merged['from_cutoff_date'].fillna(MAX_DATE)
    df_txn_merged['to_cutoff_date'] = df_txn_merged['to_cutoff_date'].fillna(MAX_DATE)
    
    # 使用向量化操作過濾，速度更快
    df_txn_cutoff = df_txn_merged[
        (df_txn_merged['txn_date'] < df_txn_merged['from_cutoff_date']) & 
        (df_txn_merged['txn_date'] < df_txn_merged['to_cutoff_date'])
    ].copy()
    
    # 清理截止日期欄位
    df_txn_cutoff.drop(columns=['from_cutoff_date', 'to_cutoff_date'], inplace=True)
    
    # 獲取所有相關帳戶 (包含發送方、接收方、提交清單中的帳戶)
    all_accts = pd.concat([
        df_txn['from_acct'], df_txn['to_acct'], df_submit['acct']
    ]).drop_duplicates().reset_index(drop=True).to_frame(name='acct')

    
    # --- 3.2 基礎交易統計 (Base Aggregation) ---
    
    # 發送統計 (Send Features)
    df_send_features = (
        df_txn_cutoff.groupby('from_acct')['txn_amt']
        .agg(
            max_send_amt='max', min_send_amt='min', avg_send_amt='mean',
            total_send_amt='sum', send_count='count'
        )
    ).reset_index().rename(columns={'from_acct': 'acct'})

    # 接收統計 (Receive Features) - **基於已截止的數據**
    df_recv_features = (
        df_txn_cutoff.groupby('to_acct')['txn_amt']
        .agg(
            max_recv_amt='max', min_recv_amt='min', avg_recv_amt='mean',
            total_recv_amt='sum', recv_count='count'
        )
    ).reset_index().rename(columns={'to_acct': 'acct'})


    # --- 3.3 熵特徵 (Entropy Features) ---
    
    # 發送方對接收方的熵 (衡量交易對手多樣性)
    df_receiver_entropy = (
        df_txn_cutoff.groupby('from_acct')['to_acct']
        .apply(calculate_entropy)
        .rename('sender_receiver_entropy')
        .reset_index().rename(columns={'from_acct': 'acct'})
    )

    # 發送方對渠道的熵 (衡量渠道使用多樣性)
    df_channel_entropy = (
        df_txn_cutoff.groupby('from_acct')['channel_type']
        .apply(calculate_entropy)
        .rename('sender_channel_entropy')
        .reset_index().rename(columns={'from_acct': 'acct'})
    )
    
    
    # --- 3.4 時段與渠道比例特徵 (Hourly & Channel Ratios) ---
    
    # 🌟 建議優化點 2：時段特徵調整 🌟
    # 根據 EDA 結果，警示比最高的時段是 4, 1, 23, 3 
    # 為了捕捉這些異常時段的交易，我們使用這些小時作為特徵
    EARLY_MORNING_HOUR = [1, 3, 4, 23] # 調整為 EDA 發現的高風險時段
    df_hourly = (
        df_txn_cutoff.groupby('from_acct')
        .agg(
            high_risk_hour_count=('hour', lambda x: (x.isin(EARLY_MORNING_HOUR)).sum()),
        )
    ).reset_index().rename(columns={'from_acct': 'acct'})
    
    # 合併筆數，計算比例
    df_hourly = df_hourly.merge(df_send_features[['acct', 'send_count']], on='acct', how='left')
    df_hourly['high_risk_hour_ratio'] = df_hourly['high_risk_hour_count'] / df_hourly['send_count'].replace(0, np.nan)
    df_hourly.drop(columns=['high_risk_hour_count', 'send_count'], inplace=True)
    
    
    # 渠道比例 One-Hot Encoding (邏輯保留，命名更清晰)
    df_channel_ratio_raw = (
        df_txn_cutoff.groupby('from_acct')['channel_type']
        .value_counts(normalize=True).rename('channel_proportion')
        .reset_index()
    )
    df_channel_ratio = df_channel_ratio_raw.pivot_table(
        index='from_acct', columns='channel_type', values='channel_proportion'
    ).fillna(0).reset_index().rename(columns={'from_acct': 'acct'})
    
    # 確保欄位名稱符合模型預期 (例如 channel_1_ratio, channel_2_ratio 等)
    df_channel_ratio.columns = [
        f'channel_{int(c)}_ratio' if isinstance(c, (int, float)) and not np.isnan(c) and c != 'acct' else c 
        for c in df_channel_ratio.columns
    ]
    
    # 低價值交易比例 (<1K Smurfing Ratio)
    LOW_VALUE_THRESHOLD = 1000
    df_low_value_count = (
        df_txn_cutoff[df_txn_cutoff['txn_amt'] < LOW_VALUE_THRESHOLD]
        .groupby('from_acct')['txn_amt'].count().rename('low_value_count')
        .reset_index().rename(columns={'from_acct': 'acct'})
    )
    df_low_value_ratio = df_low_value_count.merge(
        df_send_features[['acct', 'send_count']], on='acct', how='left'
    )
    df_low_value_ratio['low_value_txn_ratio'] = (
        df_low_value_ratio['low_value_count'] / df_low_value_ratio['send_count'].replace(0, np.nan)
    )
    df_low_value_ratio.drop(columns=['low_value_count', 'send_count'], inplace=True)


    # --- 3.5 滾動窗口特徵 (7天 / 30天快照) ---
    
    WINDOWS = [7, 30]
    df_rolling_features = all_accts[['acct']].copy()

    for window in WINDOWS:
        # 🌟 建議優化點 3：滾動窗口的實際應用 🌟
        # 由於我們已經實施了 Temporal Cutoff，這裡計算的滾動窗口
        # 其實是相對於「警示日期前的最後一筆交易」的窗口。
        # 如果要計算相對于每個帳戶的「截止日」的窗口，需要更複雜的 Grouped Time Series Aggregation
        # 但在確保截止的 df_txn_cutoff 上，以下簡單聚合仍是有效的特徵。
        
        def roll_window_agg(group, window):
            # 基於最後一筆交易日期的簡單窗口
            latest_date = group['txn_date'].max() 
            start_date = latest_date - window # window 已經是天數 (7, 30)
            window_data = group[group['txn_date'] > start_date] 
            
            return pd.Series({
                f'txn_count_{window}d': window_data.shape[0],
                f'total_amt_{window}d': window_data['txn_amt'].sum(),
                f'avg_amt_{window}d': window_data['txn_amt'].mean()
            })

        # 這裡需要傳遞 window 參數，將 apply 內的 lambda 修正
        df_win = (
            df_txn_cutoff.groupby('from_acct')
            .apply(lambda x: roll_window_agg(x, window))
            .reset_index().rename(columns={'from_acct': 'acct'})
        )
        
        df_rolling_features = df_rolling_features.merge(df_win, on='acct', how='left')


    # --- 3.6 淨流動與活動加速度特徵 (EDA Insights) ---

    df_eda_ratio = all_accts.copy()

    # 合併 total_send_amt 和 total_recv_amt
    df_eda_ratio = df_eda_ratio.merge(df_send_features[['acct', 'total_send_amt', 'send_count']], on='acct', how='left').fillna(0)
    df_eda_ratio = df_eda_ratio.merge(df_recv_features[['acct', 'total_recv_amt', 'recv_count']], on='acct', how='left').fillna(0)

    # 🌟 核心特徵：淨流動金額 (Net Flow Amount) 🌟
    df_eda_ratio['net_flow_amt'] = df_eda_ratio['total_recv_amt'] - df_eda_ratio['total_send_amt'] 

    # 🌟 核心特徵：淨流動比 (Net Flow Ratio) 🌟
    df_eda_ratio['net_flow_ratio'] = (
        df_eda_ratio['total_recv_amt'] - df_eda_ratio['total_send_amt'] 
    ) / (
        df_eda_ratio['total_send_amt'] + df_eda_ratio['total_recv_amt'] + 1
    ) # +1 防止除以零

    # 交易活動加速度 (Velocity)
    df_velocity = df_rolling_features[['acct', 'txn_count_7d', 'txn_count_30d']].copy()
    df_velocity['txn_velocity'] = (
        df_velocity['txn_count_7d'] / (df_velocity['txn_count_30d'] + 1) # +1 防止除以零
    )
    df_velocity.drop(columns=['txn_count_7d', 'txn_count_30d'], inplace=True)
    
    # 合併所有 EDA 相關特徵
    df_eda_ratio = df_eda_ratio.merge(df_velocity, on='acct', how='left')
    df_eda_ratio.drop(columns=['total_send_amt', 'total_recv_amt', 'send_count', 'recv_count'], inplace=True)


    # --- 3.7 最終特徵合併與清理 ---
    
    # 以 all_accts 為基礎合併所有特徵
    df_X = all_accts.copy()
    
    df_X = df_X.merge(df_send_features, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_recv_features, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_receiver_entropy, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_channel_entropy, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_hourly, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_channel_ratio, on='acct', how='left').fillna(0)
    df_X = df_X.merge(df_low_value_ratio, on='acct', how='left').fillna(0) 
    df_X = df_X.merge(df_rolling_features.drop(columns=['txn_count_30d', 'txn_count_7d']), on='acct', how='left').fillna(0) 
    df_X = df_X.merge(df_eda_ratio, on='acct', how='left').fillna(0)
    
    # 補上帳戶類型 (is_esun) - 關鍵的靜態特徵
    df_from = df_txn[['from_acct', 'from_acct_type']].rename(columns={'from_acct':'acct','from_acct_type':'is_esun'})
    df_to = df_txn[['to_acct', 'to_acct_type']].rename(columns={'to_acct':'acct','to_acct_type':'is_esun'})
    df_acc = pd.concat([df_from, df_to], ignore_index=True).drop_duplicates(subset=['acct']).reset_index(drop=True)
    df_X = pd.merge(df_X, df_acc, on='acct', how='left').fillna(-1) 

    print("(Finish) Advanced Feature Engineering.")

    # --- 儲存快取 ---
    df_X.to_pickle(CACHE_FILE_PATH)
    print(f"(Caching) 已將 df_X 儲存至快取檔案：{CACHE_FILE_PATH}")

(Caching) 快取檔案未找到，正在執行完整的特徵工程...


C:\Users\USER\AppData\Local\Temp\ipykernel_58584\3136033161.py:191: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: roll_window_agg(x, window))
C:\Users\USER\AppData\Local\Temp\ipykernel_58584\3136033161.py:191: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: roll_window_agg(x, window))


(Finish) Advanced Feature Engineering.
(Caching) 已將 df_X 儲存至快取檔案：df_X_features_cache.pkl


In [ ]:
# ===== 4️⃣ Train/Test Split & Ensemble Modeling (5 Models) =====
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# ----------------------------------------------------
# 1. 準備數據與分層劃分
# ----------------------------------------------------
df_X['label'] = df_X['acct'].isin(df_alert['acct']).astype(int)

X_train = df_X[(~df_X['acct'].isin(df_test['acct'])) & (df_X['is_esun']==1)].copy()
y_train = X_train['label']
X_test = df_X[df_X['acct'].isin(df_test['acct'])].copy()

feature_cols = [
    col for col in X_train.columns 
    if col not in ['acct', 'is_esun', 'label', 'anomaly_score']
]
X_train_features = X_train[feature_cols].fillna(0)
X_test_features = X_test[feature_cols].fillna(0)

# 標準化（對 Logistic Regression / SVM 效果更好）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.transform(X_test_features)

# ----------------------------------------------------
# 2. 處理類別不平衡
# ----------------------------------------------------
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight_value = neg_count / pos_count 
print(f"訓練集樣本數：總數={len(y_train)}, 正常(0)={neg_count}, 警示(1)={pos_count}")
print(f"建議權重 (scale_pos_weight)：{scale_pos_weight_value:.2f}")

# ----------------------------------------------------
# 3. K-Fold Ensemble 訓練
# ----------------------------------------------------
N_SPLITS = 5
kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_preds = np.zeros(X_train_features.shape[0])
test_preds = np.zeros(X_test_features.shape[0])

print(f"\n--- 啟動 {N_SPLITS}-Fold Ensemble 模型訓練 ---")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_features, y_train)):
    print(f"Fold {fold+1}/{N_SPLITS} ...")

    X_tr, X_val = X_train_features.iloc[train_idx], X_train_features.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    X_tr_s, X_val_s = X_train_scaled[train_idx], X_train_scaled[val_idx]

    # ----- 各模型初始化 -----
    model_lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
    model_svm = LinearSVC(class_weight='balanced', random_state=42)
    model_dt = DecisionTreeClassifier(max_depth=8, class_weight='balanced', random_state=42)
    model_xgb = xgb.XGBClassifier(
        objective='binary:logistic', eval_metric='auc', 
        scale_pos_weight=scale_pos_weight_value,
        random_state=42, n_estimators=1000, learning_rate=0.05,
        max_depth=6, subsample=0.8, colsample_bytree=0.8
    )
    model_lgb = lgb.LGBMClassifier(
        objective='binary', metric='auc',
        scale_pos_weight=scale_pos_weight_value,
        random_state=42, n_estimators=1000, learning_rate=0.05,
        num_leaves=32, max_depth=8, subsample=0.8, colsample_bytree=0.8
    )

    # ----- 模型訓練 -----
    model_lr.fit(X_tr_s, y_tr)
    model_svm.fit(X_tr_s, y_tr)
    model_dt.fit(X_tr, y_tr)
    model_xgb.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    model_lgb.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

    # ----- 驗證集預測 -----
    pred_lr = model_lr.predict_proba(X_val_s)[:,1]
    pred_svm = model_svm.decision_function(X_val_s)
    pred_svm = (pred_svm - pred_svm.min()) / (pred_svm.max() - pred_svm.min() + 1e-6)  # 正規化為機率
    pred_dt = model_dt.predict_proba(X_val)[:,1]
    pred_xgb = model_xgb.predict_proba(X_val)[:,1]
    pred_lgb = model_lgb.predict_proba(X_val)[:,1]

    # ----- 取五模型平均機率 -----
    pred_avg = (pred_lr + pred_svm + pred_dt + pred_xgb + pred_lgb) / 5
    oof_preds[val_idx] = pred_avg

    # ----- 測試集預測平均 -----
    test_lr = model_lr.predict_proba(X_test_scaled)[:,1]
    test_svm = model_svm.decision_function(X_test_scaled)
    test_svm = (test_svm - test_svm.min()) / (test_svm.max() - test_svm.min() + 1e-6)
    test_dt = model_dt.predict_proba(X_test_features)[:,1]
    test_xgb = model_xgb.predict_proba(X_test_features)[:,1]
    test_lgb = model_lgb.predict_proba(X_test_features)[:,1]

    test_preds += (test_lr + test_svm + test_dt + test_xgb + test_lgb) / (5 * N_SPLITS)

print("\n--- Ensemble 訓練完成 ---")

# ----------------------------------------------------
# 4. 尋找最大化 F1-score 的最佳閾值
# ----------------------------------------------------
precision, recall, thresholds = precision_recall_curve(y_train, oof_preds)
fscore = (2 * precision * recall) / (precision + recall)
fscore[np.isnan(fscore)] = 0
ix = np.argmax(fscore)
best_f1 = fscore[ix]
best_threshold = thresholds[ix - 1] if ix > 0 else thresholds[ix]
print(f"\n最佳 F1-score: {best_f1:.4f}, 最佳閾值: {best_threshold:.4f}")

# ----------------------------------------------------
# 5. OOF 評估與最終預測
# ----------------------------------------------------
y_oof_pred = (oof_preds >= best_threshold).astype(int)
print("\n--- OOF 混淆矩陣 ---")
print(confusion_matrix(y_train, y_oof_pred))
print("\n--- OOF 分類報告 ---")
print(classification_report(y_train, y_oof_pred, target_names=['Normal (0)', 'Alert (1)'], zero_division=0))

# 最終測試集預測
y_pred = (test_preds >= best_threshold).astype(int)
print("\n(Finish) Ensemble Modeling with F1 Optimization 完成！")


訓練集樣本數：總數=328988, 正常(0)=327984, 警示(1)=1004
LGBM 權重 (scale_pos_weight)：326.68

--- 3.1 啟動 5 折 Stratified K-Fold 訓練 (確保警示比例一致) ---
Fold 1/5...
[LightGBM] [Info] Number of positive: 803, number of negative: 262387
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008386 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6636
[LightGBM] [Info] Number of data points in the train set: 263190, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003051 -> initscore=-5.789221
[LightGBM] [Info] Start training from score -5.789221
Fold 2/5...
[LightGBM] [Info] Number of positive: 803, number of negative: 262387
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6633
[LightGBM] [Info] Numbe

In [ ]:
# ===== 5️⃣ Output CSV =====
df_pred = pd.DataFrame({
    'acct': X_test['acct'].values,
    'label': y_pred 
})

df_out = df_test[['acct']].merge(df_pred, on='acct', how='left')
out_path = "result_advanced_eda.csv"
df_out.to_csv(out_path, index=False)
print(f"(Finish) Output saved to {out_path}")

(Finish) Output saved to result_advanced_eda.csv


In [ ]:
X_tr_s

In [ ]:
y_tr

In [ ]:
X_val_s